# 연구 2단계 (GPU 버전): 3차 함수 환경에서 노이즈와 GMM 복잡도 관계 분석

## ⚠️ 실행 전 필수 확인 사항 ⚠️
본 노트북은 GPU 가속을 위해 NVIDIA RAPIDS `cuML` 라이브러리를 사용합니다. 실행을 위해서는 NVIDIA GPU 및 관련 `conda` 환경 설정이 필요합니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cuml.mixture import GaussianMixture # sklearn 대신 cuML 사용
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

## 1. 실험 조건 설정

In [ ]:
N_RUNS = 30
NOISE_LEVELS = [0.1, 1, 5, 10, 20, 50]
N_COMPONENTS_RANGE = range(1, 26)
N_SAMPLES = 1000

## 2. 데이터 생성 및 BIC 계산 함수 정의

In [ ]:
def generate_3d_data(n_samples, noise_level):
    x = np.linspace(-10, 10, n_samples)
    y = 0.05 * x**3 - 0.5 * x**2 + 0.1*x + 5
    y_noisy = y + np.random.randn(n_samples) * noise_level
    # cuML은 float32 타입을 선호
    return np.vstack([x, y_noisy]).T.astype(np.float32)

def calculate_bic(gmm, data):
    """cuML GMM 모델에 대한 BIC 점수를 수동으로 계산합니다."""
    n_samples, n_features = data.shape
    n_components = gmm.n_components
    
    cov_params = n_components * n_features * (n_features + 1) / 2.
    mean_params = n_components * n_features
    weight_params = n_components - 1
    n_params = cov_params + mean_params + weight_params
    
    log_likelihood = gmm.score(data)
    bic = n_params * np.log(n_samples) - 2 * log_likelihood * n_samples
    return bic

## 3. 핵심 연구 로직 실행 (GPU 가속)

In [ ]:
results = []

for noise in tqdm(NOISE_LEVELS, desc="Noise Levels"):
    optimal_k_for_runs = []
    for run in tqdm(range(N_RUNS), desc=f"Runs for Noise {noise}", leave=False):
        data = generate_3d_data(N_SAMPLES, noise)
        
        bics = []
        for k in N_COMPONENTS_RANGE:
            gmm = GaussianMixture(n_components=k, reg_covar=1e-6)
            gmm.fit(data)
            bic_score = calculate_bic(gmm, data)
            bics.append(bic_score)
            
        best_k = N_COMPONENTS_RANGE[np.argmin(bics)]
        optimal_k_for_runs.append(best_k)
        
    results.append({
        'noise_level': noise,
        'mean_k_opt': np.mean(optimal_k_for_runs),
        'std_k_opt': np.std(optimal_k_for_runs)
    })

results_df_3d = pd.DataFrame(results)

## 4. 결과 분석 및 시각화

In [ ]:
print("--- 3차 함수 실험 결과 (GPU) ---")
print(results_df_3d[['noise_level', 'mean_k_opt', 'std_k_opt']])

plt.figure(figsize=(12, 7))
plt.errorbar(
    results_df_3d['noise_level'], 
    results_df_3d['mean_k_opt'], 
    yerr=results_df_3d['std_k_opt'],
    fmt='-s', capsize=5, label='평균 최적 k (오차막대: 1-std)'
)
plt.title('노이즈 수준에 따른 GMM 최적 컴포넌트 개수 변화 (3차 함수, GPU)', fontsize=16)
plt.xlabel('노이즈 수준 (Noise Level)', fontsize=12)
plt.ylabel(f'최적 컴포넌트 개수 (평균, N={N_RUNS}회 실행)', fontsize=12)
plt.xticks(NOISE_LEVELS)
plt.grid(True, which='both', linestyle='--')
plt.legend()
plt.show()